# Relational API

- インクリメンタルにクエリを構築
- `DuckDBPyRelation` ノードから呼び出す
- SQLクエリをシンボリックに表現
- 実行のトリガーとなるメソッドが呼び出されるまで、リレーションはデータを保持・実行しない

## Relationの作成

- `sql`
- `from_arrow`
- `from_df`
- `read_csv`
- `read_json`
- `read_parquet`

In [1]:
import duckdb

cpi = duckdb.sql(
    "SELECT * FROM 'https://pub-5c988f48e21d45ec95c1e3eca8ab1787.r2.dev/0003427113.parquet'"
)
cpi.show()

┌──────────┬──────────────────────────┬────────────┬───────────────────────────────────────────┬───────────┬────────────────────┬────────────┬──────────────────┬─────────┬────────┬────────────┐
│ tab_code │         表章項目         │ cat01_code │              2020年基準品目               │ area_code │ 地域（2020年基準） │ time_code  │ 時間軸（年・月） │  unit   │ value  │ annotation │
│  int64   │         varchar          │   int64    │                  varchar                  │  varchar  │      varchar       │   int64    │     varchar      │ varchar │ double │   double   │
├──────────┼──────────────────────────┼────────────┼───────────────────────────────────────────┼───────────┼────────────────────┼────────────┼──────────────────┼─────────┼────────┼────────────┤
│        1 │ 指数                     │          1 │ 0001 総合                                 │ 13A01     │ 13100 東京都区部   │ 1970000303 │ 1970年3月        │ NULL    │   31.2 │       NULL │
│        1 │ 指数                     │          1 │ 0001 総合       

```{note}
- 13009354のデータを計算するRelationsが即座に構築されている
- サマリの表示（ `show()` ）が即座に出力される
```

Relationは `FROM` 句から参照できる

In [2]:
tab_code = duckdb.sql("SELECT DISTINCT(tab_code) from cpi")
tab_code.show()

┌──────────┐
│ tab_code │
│  int64   │
├──────────┤
│        1 │
│        3 │
│        2 │
└──────────┘



## オペレーション

- Relationからメソッドを実行してデータを操作できる
- 実行結果はRelationで返される

集約関数を適用:

`aggregate(expr, groups = {})`

In [3]:
cpi.aggregate("min(tab_code), max(tab_code)")

┌───────────────┬───────────────┐
│ min(tab_code) │ max(tab_code) │
│     int64     │     int64     │
├───────────────┼───────────────┤
│             1 │             3 │
└───────────────┴───────────────┘

式を適用

`project(expr)`

In [4]:
tab_code.project("tab_code + 100")

┌──────────────────┐
│ (tab_code + 100) │
│      int64       │
├──────────────────┤
│              102 │
│              103 │
│              101 │
└──────────────────┘

n行を選択

`limit(n, offset = 0)`

In [5]:
tab_code.limit(2)

┌──────────┐
│ tab_code │
│  int64   │
├──────────┤
│        2 │
│        1 │
└──────────┘

ソート

`order(expr)`

In [6]:
tab_code.order("tab_code DESC")

┌──────────┐
│ tab_code │
│  int64   │
├──────────┤
│        3 │
│        2 │
│        1 │
└──────────┘

元のRelationにあり、relに存在しない行

`except_(rel)`

In [7]:
tab_code.except_(tab_code.limit(2))

┌──────────┐
│ tab_code │
│  int64   │
├──────────┤
│        2 │
└──────────┘

両方に存在する行

`intersect(rel)`

In [8]:
tab_code.intersect(tab_code.limit(2))

┌──────────┐
│ tab_code │
│  int64   │
├──────────┤
│        3 │
│        2 │
└──────────┘

結合

`union(rel)`

In [9]:
tab_code.union(tab_code.limit(2))

┌──────────┐
│ tab_code │
│  int64   │
├──────────┤
│        3 │
│        1 │
│        2 │
│        3 │
│        2 │
└──────────┘

条件に満たす行を抽出

`filter(condition)`

In [10]:
tab_code.filter("tab_code > 2")

┌──────────┐
│ tab_code │
│  int64   │
├──────────┤
│        3 │
└──────────┘

条件に基づいた結合

`join(rel, condition, type = "inner")`

In [11]:
r1 = duckdb.sql(
    "SELECT area_code, cat01_code, value FROM cpi WHERE time_code = '1970000303' AND cat01_code = 1"
).set_alias("r1")
r2 = duckdb.sql(
    "SELECT area_code, cat01_code, value FROM cpi WHERE time_code = '1970000404' AND cat01_code = 1"
).set_alias("r2")

In [12]:
r1.join(r2, "r1.area_code = r2.area_code")

┌───────────┬────────────┬────────┬───────────┬────────────┬────────┐
│ area_code │ cat01_code │ value  │ area_code │ cat01_code │ value  │
│  varchar  │   int64    │ double │  varchar  │   int64    │ double │
├───────────┼────────────┼────────┼───────────┼────────────┼────────┤
│ 13A01     │          1 │   31.2 │ 13A01     │          1 │    0.6 │
│ 00000     │          1 │   30.6 │ 00000     │          1 │    0.9 │
│ 00012     │          1 │   30.8 │ 00012     │          1 │    0.8 │
│ 00013     │          1 │   30.9 │ 00013     │          1 │    1.1 │
│ 00018     │          1 │   30.3 │ 00018     │          1 │    0.9 │
│ 00042     │          1 │   32.0 │ 00042     │          1 │    0.7 │
│ 00043     │          1 │   30.0 │ 00043     │          1 │    0.8 │
│ 00044     │          1 │   30.7 │ 00044     │          1 │    0.6 │
│ 00049     │          1 │   30.5 │ 00049     │          1 │    1.3 │
│ 00050     │          1 │   30.7 │ 00050     │          1 │    0.7 │
│   ·       │       

または

```{seealso}
`ColumnExpression` :

[](./expression-api.ipynb)
```

In [13]:
cols = [
    duckdb.ColumnExpression("area_code"),
    duckdb.ColumnExpression("cat01_code"),
    duckdb.ColumnExpression("value"),
]
r1 = (
    duckdb.sql("SELECT * FROM cpi")
    .filter("time_code = '1970000303' AND cat01_code = 1")
    .select(*cols)
).set_alias("r1")
r2 = (
    duckdb.sql("SELECT * FROM cpi")
    .filter("time_code = '1970000404' AND cat01_code = 1")
    .select(*cols)
).set_alias("r2")
r1.join(r2, "r1.area_code = r2.area_code")

┌───────────┬────────────┬────────┬───────────┬────────────┬────────┐
│ area_code │ cat01_code │ value  │ area_code │ cat01_code │ value  │
│  varchar  │   int64    │ double │  varchar  │   int64    │ double │
├───────────┼────────────┼────────┼───────────┼────────────┼────────┤
│ 13A01     │          1 │   31.2 │ 13A01     │          1 │    0.6 │
│ 00000     │          1 │   30.6 │ 00000     │          1 │    0.9 │
│ 00012     │          1 │   30.8 │ 00012     │          1 │    0.8 │
│ 00013     │          1 │   30.9 │ 00013     │          1 │    1.1 │
│ 00018     │          1 │   30.3 │ 00018     │          1 │    0.9 │
│ 00042     │          1 │   32.0 │ 00042     │          1 │    0.7 │
│ 00043     │          1 │   30.0 │ 00043     │          1 │    0.8 │
│ 00044     │          1 │   30.7 │ 00044     │          1 │    0.6 │
│ 00049     │          1 │   30.5 │ 00049     │          1 │    1.3 │
│ 00050     │          1 │   30.7 │ 00050     │          1 │    0.7 │
│   ·       │       